# Section: Encrypted Deep Learning

- Lesson: Reviewing Additive Secret Sharing
- Lesson: Encrypted Subtraction and Public/Scalar Multiplication
- Lesson: Encrypted Computation in PySyft
- Project: Build an Encrypted Database
- Lesson: Encrypted Deep Learning in PyTorch
- Lesson: Encrypted Deep Learning in Keras
- Final Project

# Lesson: Reviewing Additive Secret Sharing

_For more great information about SMPC protocols like this one, visit https://mortendahl.github.io. With permission, Morten's work directly inspired this first teaching segment._

In [0]:
import random
import numpy as np

BASE = 10

PRECISION_INTEGRAL = 8
PRECISION_FRACTIONAL = 8
Q = 293973345475167247070445277780365744413

PRECISION = PRECISION_INTEGRAL + PRECISION_FRACTIONAL

assert(Q > BASE**PRECISION)

def encode(rational):
    upscaled = int(rational * BASE**PRECISION_FRACTIONAL)
    field_element = upscaled % Q
    return field_element

def decode(field_element):
    upscaled = field_element if field_element <= Q/2 else field_element - Q
    rational = upscaled / BASE**PRECISION_FRACTIONAL
    return rational

def encrypt(secret):
    first  = random.randrange(Q)
    second = random.randrange(Q)
    third  = (secret - first - second) % Q
    return [first, second, third]

def decrypt(sharing):
    return sum(sharing) % Q

def add(a, b):
    c = list()
    for i in range(len(a)):
        c.append((a[i] + b[i]) % Q)
    return tuple(c)

In [0]:
x = encrypt(encode(5.5))
x

[58783099858366721685668308921263689881,
 192008164583996253091925146547807143892,
 43182081032804272292851822311844910640]

In [0]:
y = encrypt(encode(2.3))
y

[268158955783880865753951359585838906285,
 124858285930651164494151441112566089917,
 194929449235802463892787754862556492623]

In [0]:
z = add(x,y)
z

(32968710167080340369174390726736851753,
 22893105039480170515631309880007489396,
 238111530268606736185639577174401403263)

In [0]:
decode(decrypt(z))

7.79999999

# Lesson: Encrypted Subtraction and Public/Scalar Multiplication

In [0]:
field = 23740629843760239486723

In [0]:
x = 5

bob_x_share = 2372385723 # random number
alices_x_share = field - bob_x_share + x

In [0]:
(bob_x_share + alices_x_share) % field

5

In [0]:
field = 10

x = 5

bob_x_share = 8
alice_x_share = field - bob_x_share + x

y = 1

bob_y_share = 9
alice_y_share = field - bob_y_share + y

In [0]:
((bob_x_share + alice_x_share) - (bob_y_share + alice_y_share)) % field

4

In [0]:
((bob_x_share - bob_y_share) + (alice_x_share - alice_y_share)) % field

4

In [0]:
bob_x_share + alice_x_share + bob_y_share + alice_y_share

26

In [0]:
bob_z_share = (bob_x_share - bob_y_share)
alice_z_share = (alice_x_share - alice_y_share)

In [0]:
(bob_z_share + alice_z_share) % field

4

In [0]:
def sub(a, b):
    c = list()
    for i in range(len(a)):
        c.append((a[i] - b[i]) % Q)
    return tuple(c)

In [0]:
field = 10

x = 5

bob_x_share = 8
alice_x_share = field - bob_x_share + x

y = 1

bob_y_share = 9
alice_y_share = field - bob_y_share + y

In [0]:
bob_x_share + alice_x_share

15

In [0]:
bob_y_share + alice_y_share

11

In [0]:
((bob_y_share * 3) + (alice_y_share * 3)) % field

3

In [0]:
def imul(a, scalar):
    
    # logic here which can multiply by a public scalar
    
    c = list()
    
    for i in range(len(a)):
        c.append((a[i] * scalar) % Q)
        
    return tuple(c)

In [0]:
x = encrypt(encode(5.5))
x

[242034884016359953980383331990732470321,
 235649927039650326824613430138335805994,
 110261879894324213335893793432213212511]

In [0]:
z = imul(x, 3)

In [0]:
decode(decrypt(z))

16.5

# Lesson: Encrypted Computation in PySyft

In [0]:
# pip install syft

In [0]:
import syft as sy
import torch as th
hook = sy.TorchHook(th)
from torch import nn, optim

W0726 17:14:24.492445 139942056519552 hook.py:98] Torch was already hooked... skipping hooking process


In [0]:
bob = sy.VirtualWorker(hook, id="bob").add_worker(sy.local_worker)
alice = sy.VirtualWorker(hook, id="alice").add_worker(sy.local_worker)
secure_worker = sy.VirtualWorker(hook, id="secure_worker").add_worker(sy.local_worker)

W0726 17:14:24.508656 139942056519552 base.py:628] Worker me already exists. Replacing old worker which could cause                     unexpected behavior
W0726 17:14:24.511101 139942056519552 base.py:628] Worker me already exists. Replacing old worker which could cause                     unexpected behavior
W0726 17:14:24.513001 139942056519552 base.py:628] Worker me already exists. Replacing old worker which could cause                     unexpected behavior


In [0]:
x = th.tensor([1,2,3,4])
y = th.tensor([2,-1,1,0])

In [0]:
x = x.share(bob, alice, crypto_provider=secure_worker)

In [0]:
y = y.share(bob, alice, crypto_provider=secure_worker)

In [0]:
z = x + y
z.get()

tensor([3, 1, 4, 4])

In [0]:
z = x - y
z.get()

tensor([-1,  3,  2,  4])

In [0]:
z = x * y
z.get()

tensor([ 2, -2,  3,  0])

In [0]:
z = x > y
z.get()

tensor([0, 1, 1, 1])

In [0]:
z = x < y
z.get()

tensor([1, 0, 0, 0])

In [0]:
z = x == y
z.get()

tensor([0, 0, 0, 0])

In [0]:
x = th.tensor([1,2,3,4])
y = th.tensor([2,-1,1,0])

x = x.fix_precision().share(bob, alice, crypto_provider=secure_worker)
y = y.fix_precision().share(bob, alice, crypto_provider=secure_worker)

In [0]:
z = x + y
z.get().float_precision()

tensor([3., 1., 4., 4.])

In [0]:
z = x - y
z.get().float_precision()

tensor([-1.,  3.,  2.,  4.])

In [0]:
z = x * y
z.get().float_precision()

tensor([ 2., -2.,  3.,  0.])

In [0]:
z = x > y
z.get().float_precision()

tensor([0., 1., 1., 1.])

In [0]:
z = x < y
z.get().float_precision()

tensor([1., 0., 0., 0.])

In [0]:
z = x == y
z.get().float_precision()

tensor([0., 0., 0., 0.])

# Project: Build an Encrypted Database

In [0]:
# try this project here!
import string
import torch as th

In [0]:
char2index = {}
index2char = {}

In [0]:
for index,char in enumerate(" " + string.ascii_lowercase + string.digits + string.punctuation):
  char2index[char] = index
  index2char[index] = char

In [0]:
def string2value(str_input, max_len=8):
  str_input = str_input[:max_len].lower()

  if len(str_input) < max_len:
    str_input = str_input + '.' * (max_len - len(str_input))

  value = list()
  for char in str_input:
    value.append(char2index[char])
  
  return th.tensor(value).long()

def value2string(input_value):
  result_str = ''
  for val in input_value:
    result_str += index2char[int(val)]
  result_str = result_str.replace('.',"")
  return result_str

In [0]:
string2value("hello")

tensor([ 8,  5, 12, 12, 15, 50, 50, 50])

In [0]:
def one_hot(index,length):
  tensor = th.zeros(length).long()
  tensor[index] = 1
  return tensor
  

In [0]:
def string2one_hot_matrix(str_input, max_len=8):
  str_input = str_input[:max_len].lower()

  if len(str_input) < max_len:
    str_input = str_input + '.' * (max_len - len(str_input))

  value = list()
  for char in str_input:
    value.append(one_hot(char2index[char],len(char2index)).unsqueeze(0))

  return th.cat(value,0)


In [0]:
def string_equal(string_a, string_b):
  vec = (string_a * string_b).sum(1)
  x = vec[0]
  for i in range(vec.shape[0]-1):
    x = x * vec[i+1]
  return x

In [0]:
matrix = string2one_hot_matrix("hello")

In [0]:
string_a = string2one_hot_matrix("Hello")
string_b = string2one_hot_matrix("Hello")

In [0]:
class EncryptedDB():
  def __init__(self, *owners, max_key_length, max_value_length):
    
    self.keys = list()
    self.values = list()
    
    self.owners = owners
    
    self.max_key_length = max_key_length
    self.max_value_length = max_value_length
    
  def add_entry(self, key, value):
    key = string2one_hot_matrix(key,self.max_key_length)
    key = key.share(*self.owners)
    self.keys.append(key)
    
    value = string2value(value,self.max_value_length)
    value = value.share(*self.owners)
    self.values.append(value)
    
  def query(self,query_str):
    query_matrix = string2one_hot_matrix(query_str)
    query_matrix = query_matrix.share(*self.owners)
    
    match_keys = list()
    for key in self.keys:
      match_keys.append(string_equal(key,query_matrix))

    result = self.values[0] * match_keys[0]
    for i in range(len(self.values) - 1 ):
      result += self.values[i + 1] * match_keys[i + 1]
    result = result.get()
    
    return value2string(result)



In [0]:
db = EncryptedDB(bob,alice,secure_worker,max_key_length=8,max_value_length=8)

In [0]:
db.add_entry('key1','value1')
db.add_entry('key2','value2')
db.add_entry('key3','value3')
db.add_entry('key4','value4')
db.add_entry('key5','value5')

In [0]:
query_str = "key5"
db.query(query_str)

'value5'

In [0]:
db.values

[(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:46029003698 -> bob:66256141534]
 	-> (Wrapper)>[PointerTensor | me:59144208925 -> alice:29478995270]
 	-> (Wrapper)>[PointerTensor | me:33892320620 -> secure_worker:28450775982]
 	*crypto provider: me*, (Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:49332750539 -> bob:20213900507]
 	-> (Wrapper)>[PointerTensor | me:79958765513 -> alice:54703066843]
 	-> (Wrapper)>[PointerTensor | me:739500434 -> secure_worker:40047962341]
 	*crypto provider: me*, (Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:34671626999 -> bob:94016123014]
 	-> (Wrapper)>[PointerTensor | me:13522200000 -> alice:10840074907]
 	-> (Wrapper)>[PointerTensor | me:96918787966 -> secure_worker:42135056651]
 	*crypto provider: me*, (Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:57246001234 -> bob:36954326009]
 	-> (Wrapper)>[PointerTensor | me:309522354 -> alice:24269528204]
 	-> (Wrapper)>[P

# Lesson: Encrypted Deep Learning in PyTorch

### Train a Model

In [1]:
import torch as th
from torch import nn
from torch import optim
import torch.nn.functional as F

# A Toy Dataset
data = th.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = th.tensor([[0],[0],[1],[1.]], requires_grad=True)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 20)
        self.fc2 = nn.Linear(20, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

# A Toy Model
model = Net()

def train():
    # Training Logic
    opt = optim.SGD(params=model.parameters(),lr=0.1)
    for iter in range(20):

        # 1) erase previous gradients (if they exist)
        opt.zero_grad()

        # 2) make a prediction
        pred = model(data)

        # 3) calculate how much we missed
        loss = ((pred - target)**2).sum()

        # 4) figure out which weights caused us to miss
        loss.backward()

        # 5) change those weights
        opt.step()

        # 6) print our progress
        print(loss.data)
        
train()

tensor(2.9420)
tensor(28.7438)
tensor(23.5629)
tensor(1.2654)
tensor(0.9525)
tensor(0.9256)
tensor(0.9067)
tensor(0.8852)
tensor(0.8603)
tensor(0.8321)
tensor(0.8007)
tensor(0.7578)
tensor(0.6507)
tensor(0.5309)
tensor(0.4056)
tensor(0.2902)
tensor(0.1983)
tensor(0.1339)
tensor(0.0940)
tensor(0.0687)


In [2]:
model(data)

tensor([[0.0052],
        [0.1945],
        [1.0384],
        [0.8883]], grad_fn=<AddmmBackward>)

## Encrypt the Model and Data

In [3]:
# !pip install syft
import syft as sy
hook = sy.TorchHook(th)

W0727 13:07:01.627888 139906829059968 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0727 13:07:01.650239 139906829059968 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [0]:
bob = sy.VirtualWorker(hook,id="bob")
alice = sy.VirtualWorker(hook,id="alice")
secure_worker = sy.VirtualWorker(hook,id="secure_worker")

In [0]:
encrypted_model = model.fix_precision().share(alice, bob, crypto_provider=secure_worker)

In [6]:
list(encrypted_model.parameters())

[Parameter containing:
 Parameter>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:97281737290 -> alice:84700623885]
 	-> (Wrapper)>[PointerTensor | me:62551344033 -> bob:81805484208]
 	*crypto provider: secure_worker*, Parameter containing:
 Parameter>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:62598448262 -> alice:30930260768]
 	-> (Wrapper)>[PointerTensor | me:93888728527 -> bob:27260042062]
 	*crypto provider: secure_worker*, Parameter containing:
 Parameter>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:18952021592 -> alice:88177925452]
 	-> (Wrapper)>[PointerTensor | me:4647676561 -> bob:79635673466]
 	*crypto provider: secure_worker*, Parameter containing:
 Parameter>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:87514919692 -> alice:55695613666]
 	-> (Wrapper)>[PointerTensor | me:71170257853 -> bob:249830254

In [0]:
encrypted_data = data.fix_precision().share(alice, bob, crypto_provider=secure_worker)

In [8]:
encrypted_data

(Wrapper)>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:84534694572 -> alice:62965111261]
	-> (Wrapper)>[PointerTensor | me:13854757232 -> bob:86477203264]
	*crypto provider: secure_worker*

In [0]:
encrypted_prediction = encrypted_model(encrypted_data)

In [10]:
encrypted_prediction.get().float_precision()

tensor([[0.0050],
        [0.1940],
        [1.0370],
        [0.8870]])

# Lesson: Encrypted Deep Learning in Keras


## Step 1: Public Training

Welcome to this tutorial! In the following notebooks you will learn how to provide private predictions. By private predictions, we mean that the data is constantly encrypted throughout the entire process. At no point is the user sharing raw data, only encrypted (that is, secret shared) data. In order to provide these private predictions, Syft Keras uses a library called [TF Encrypted](https://github.com/tf-encrypted/tf-encrypted) under the hood. TF Encrypted combines cutting-edge cryptographic and machine learning techniques, but you don't have to worry about this and can focus on your machine learning application.

You can start serving private predictions with only three steps:
- **Step 1**: train your model with normal Keras.
- **Step 2**: secure and serve your machine learning model (server).
- **Step 3**: query the secured model to receive private predictions (client). 

Alright, let's go through these three steps so you can deploy impactful machine learning services without sacrificing user privacy or model security.

Huge shoutout to the Dropout Labs ([@dropoutlabs](https://twitter.com/dropoutlabs)) and TF Encrypted ([@tf_encrypted](https://twitter.com/tf_encrypted)) teams for their great work which makes this demo possible, especially: Jason Mancuso ([@jvmancuso](https://twitter.com/jvmancuso)), Yann Dupis ([@YannDupis](https://twitter.com/YannDupis)), and Morten Dahl ([@mortendahlcs](https://github.com/mortendahlcs)). 

_Demo Ref: https://github.com/OpenMined/PySyft/tree/dev/examples/tutorials_

## Train Your Model in Keras

To use privacy-preserving machine learning techniques for your projects you should not have to learn a new machine learning framework. If you have basic [Keras](https://keras.io/) knowledge, you can start using these techniques with Syft Keras. If you have never used Keras before, you can learn a bit more about it through the [Keras documentation](https://keras.io). 

Before serving private predictions, the first step is to train your model with normal Keras. As an example, we will train a model to classify handwritten digits. To train this model we will use the canonical [MNIST dataset](http://yann.lecun.com/exdb/mnist/).

We borrow [this example](https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py) from the reference Keras repository.  To train your classification model, you just run the cell below.

In [0]:
from __future__ import print_function
import tensorflow.keras as keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, AveragePooling2D
from tensorflow.keras.layers import Activation

batch_size = 128
num_classes = 10
epochs = 2

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()

model.add(Conv2D(10, (3, 3), input_shape=input_shape))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

11493376/11490434 [==============================] - 0s 0us/step


W0726 17:55:31.516157 139942056519552 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Train on 60000 samples, validate on 10000 samples


Exception ignored in: <object repr() failed>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/syft/frameworks/torch/pointers/object_pointer.py", line 219, in __del__
    self.owner.send_msg(MSGTYPE.FORCE_OBJ_DEL, self.id_at_location, self.location)
  File "/usr/local/lib/python3.6/dist-packages/syft/workers/base.py", line 223, in send_msg
    bin_response = self._send_msg(bin_message, location)
  File "/usr/local/lib/python3.6/dist-packages/syft/workers/virtual.py", line 10, in _send_msg
    return location._recv_msg(message)
AttributeError: 'int' object has no attribute '_recv_msg'


Epoch 1/2
60000/60000 [==============================] - 21s 355us/sample - loss: 2.3054 - acc: 0.1006 - val_loss: 2.3036 - val_acc: 0.1155
Epoch 2/2
60000/60000 [==============================] - 21s 356us/sample - loss: 2.3026 - acc: 0.1150 - val_loss: 2.3009 - val_acc: 0.1324
Test loss: 2.300855921173096
Test accuracy: 0.1324


In [0]:
## Save your model's weights for future private prediction
model.save('short-conv-mnist.h5')

## Step 2: Load and Serve the Model

Now that you have a trained model with normal Keras, you are ready to serve some private predictions. We can do that using Syft Keras.

To secure and serve this model, we will need three TFEWorkers (servers). This is because TF Encrypted under the hood uses an encryption technique called [multi-party computation (MPC)](https://en.wikipedia.org/wiki/Secure_multi-party_computation). The idea is to split the model weights and input data into shares, then send a share of each value to the different servers. The key property is that if you look at the share on one server, it reveals nothing about the original value (input data or model weights).

We'll define a Syft Keras model like we did in the previous notebook. However, there is a trick: before instantiating this model, we'll run `hook = sy.KerasHook(tf.keras)`. This will add three important new methods to the Keras Sequential class:
 - `share`: will secure your model via secret sharing; by default, it will use the SecureNN protocol from TF Encrypted to secret share your model between each of the three TFEWorkers. Most importantly, this will add the capability of providing predictions on encrypted data.
 - `serve`: this function will launch a serving queue, so that the TFEWorkers can can accept prediction requests on the secured model from external clients.
 - `shutdown_workers`: once you are done providing private predictions, you can shut down your model by running this function. It will direct you to shutdown the server processes manually if you've opted to manually manage each worker.

If you want learn more about MPC, you can read this excellent [blog](https://mortendahl.github.io/2017/04/17/private-deep-learning-with-mpc/).

In [0]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import AveragePooling2D, Conv2D, Dense, Activation, Flatten, ReLU, Activation

import syft as sy
hook = sy.KerasHook(tf.keras)

## Model

As you can see, we define almost the exact same model as before, except we provide a `batch_input_shape`. This allows TF Encrypted to better optimize the secure computations via predefined tensor shapes. For this MNIST demo, we'll send input data with the shape of (1, 28, 28, 1). 
We also return the logit instead of softmax because this operation is complex to perform using MPC, and we don't need it to serve prediction requests.

In [0]:
num_classes = 10
input_shape = (1, 28, 28, 1)

model = Sequential()

model.add(Conv2D(10, (3, 3), batch_input_shape=input_shape))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(num_classes, name="logit"))

### Load Pre-trained Weights

With `load_weights` you can easily load the weights you have saved previously after training your model.

In [0]:
pre_trained_weights = 'short-conv-mnist.h5'
model.load_weights(pre_trained_weights)

## Step 3: Setup Your Worker Connectors

Let's now connect to the TFEWorkers (`alice`, `bob`, and `carol`) required by TF Encrypted to perform private predictions. For each TFEWorker, you just have to specify a host.

These workers run a [TensorFlow server](https://www.tensorflow.org/api_docs/python/tf/distribute/Server), which you can either manage manually (`AUTO = False`) or ask the workers to manage for you (`AUTO = True`). If choosing to manually manage them, you will be instructed to execute a terminal command on each worker's host device after calling `model.share()` below.  If all workers are hosted on a single device (e.g. `localhost`), you can choose to have Syft automatically manage the worker's TensorFlow server.

In [0]:
AUTO = False

alice = sy.TFEWorker(host='localhost:4000', auto_managed=AUTO)
bob = sy.TFEWorker(host='localhost:4001', auto_managed=AUTO)
carol = sy.TFEWorker(host='localhost:4002', auto_managed=AUTO)

## Step 4: Split the Model Into Shares

Thanks to `sy.KerasHook(tf.keras)` you can call the `share` method to transform your model into a TF Encrypted Keras model.

If you have asked to manually manage servers above then this step will not complete until they have all been launched. Note that your firewall may ask for Python to accept incoming connection.

In [0]:
model.share(alice, bob, carol)

INFO:tf_encrypted:If not done already, please launch the following command in a terminal on host 'localhost:4000':
'python -m tf_encrypted.player --config /tmp/tfe.config server0'
This can be done automatically in a local subprocess by setting `auto_managed=True` when instantiating a TFEWorker.
INFO:tf_encrypted:If not done already, please launch the following command in a terminal on host 'localhost:4001':
'python -m tf_encrypted.player --config /tmp/tfe.config server1'
This can be done automatically in a local subprocess by setting `auto_managed=True` when instantiating a TFEWorker.
INFO:tf_encrypted:If not done already, please launch the following command in a terminal on host 'localhost:4002':
'python -m tf_encrypted.player --config /tmp/tfe.config server2'
This can be done automatically in a local subprocess by setting `auto_managed=True` when instantiating a TFEWorker.
INFO:tf_encrypted:Starting session on target 'grpc://localhost:4000' using config graph_options {
}



## Step 5: Launch 3 Servers

```
python -m tf_encrypted.player --config /tmp/tfe.config server0
python -m tf_encrypted.player --config /tmp/tfe.config server1
python -m tf_encrypted.player --config /tmp/tfe.config server2```

## Step 6: Serve the Model

Perfect! Now by calling `model.serve`, your model is ready to provide some private predictions. You can set `num_requests` to set a limit on the number of predictions requests served by the model; if not specified then the model will be served until interrupted.

In [0]:
model.serve(num_requests=3)

Served encrypted prediction 1 to client.
Served encrypted prediction 2 to client.
Served encrypted prediction 3 to client.


## Step 7: Run the Client

At this point open up and run the companion notebook: Section 4b - Encrytped Keras Client

## Step 8: Shutdown the Servers

Once your request limit above, the model will no longer be available for serving requests, but it's still secret shared between the three workers above. You can kill the workers by executing the cell below.

**Congratulations** on finishing Part 12: Secure Classification with Syft Keras and TFE!

In [0]:
model.shutdown_workers()

if not AUTO:
    process_ids = !ps aux | grep '[p]ython -m tf_encrypted.player --config /tmp/tfe.config' | awk '{print $2}'
    for process_id in process_ids:
        !kill {process_id}
        print("Process ID {id} has been killed.".format(id=process_id))

# Keystone Project - Mix and Match What You've Learned

Description: Take two of the concepts you've learned about in this course (Encrypted Computation, Federated Learning, Differential Privacy) and combine them for a use case of your own design. Extra credit if you can get your demo working with [WebSocketWorkers](https://github.com/OpenMined/PySyft/tree/dev/examples/tutorials/advanced/websockets-example-MNIST) instead of VirtualWorkers! Then take your demo or example application, write a blogpost, and share that blogpost in #general-discussion on OpenMined's slack!!!

Inspiration:
- This Course's Code: https://github.com/Udacity/private-ai
- OpenMined's Tutorials: https://github.com/OpenMined/PySyft/tree/dev/examples/tutorials
- OpenMined's Blog: https://blog.openmined.org